- add click ratio

In [1]:
import os
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn import preprocessing
from tqdm._tqdm_notebook import tqdm_notebook
from sklearn.metrics import average_precision_score
tqdm_notebook.pandas(desc="progress: ")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  
/opt/conda/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
DIR = "../input/20201115at/"
train = pd.read_csv(DIR+"train.csv")
test = pd.read_csv(DIR+"test.csv")
campaign = pd.read_csv(DIR+"campaign.csv")

advertiser_video = pd.read_csv(DIR+"advertiser_video.csv")
map_game_feed_native_video_assets = pd.read_csv(DIR+'map_game_feed_native_video_assets.csv')

#advertiser_converted_video = pd.read_csv(DIR+"advertiser_converted_video.csv")

# preprocess

In [3]:
# with campaign
train = train.merge(campaign, left_on="campaign_id", right_on = "id", how="left").drop(["mst_advertiser_id", "id"], axis=1)
test = test.merge(campaign, left_on="campaign_id", right_on = "id", how="left").drop(["mst_advertiser_id", "id"], axis=1)

# with map_gae_feed_native_video_assets
#train = train.merge(map_game_feed_native_video_assets, left_on=["game_feed_id"], right_on = ["mst_game_feed_id"], how="left").drop(["mst_game_feed_id"], axis=1)
#test = test.merge(map_game_feed_native_video_assets, left_on=["game_feed_id"], right_on = ["mst_game_feed_id"], how="left").drop(["mst_game_feed_id"], axis=1)

# with advertiser_video horizontal
#horizontal = advertiser_video.copy()
#horizontal.columns = [f"horizontal_{c}" if c not in ["id", "mst_advertiser_id"] else c for c in horizontal.columns]
#train = pd.merge(train, horizontal[["id", "mst_advertiser_id", "horizontal_duration"]], left_on=["horizontal_mst_advertiser_video_id", "advertiser_id"], 
#               right_on=["id", "mst_advertiser_id"], how="left").drop(columns=["id", "mst_advertiser_id"])
#test = pd.merge(test, horizontal[["id", "mst_advertiser_id", "horizontal_duration"]], left_on=["horizontal_mst_advertiser_video_id", "advertiser_id"], 
#               right_on=["id", "mst_advertiser_id"], how="left").drop(columns=["id", "mst_advertiser_id"])

In [4]:
train["imp_at"] = pd.to_datetime(train["imp_at"])
test["imp_at"] = pd.to_datetime(test["imp_at"])

In [5]:
train.drop(['click_or_view_at', 'cv_at','last_cv_interval', 'last_paid_interval'], axis=1, inplace=True)

In [6]:
object_feats = [i for i in train.columns if "object" == str(train[i].dtype)]
train[object_feats] = train[object_feats].fillna("-999")
test[object_feats] = test[object_feats].fillna("-999")

for i in object_feats:
    print(i)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(pd.concat([train[i], test[i]])))
    train[i] = lbl.transform(list(train[i]))
    test[i] = lbl.transform(list(test[i]))

request_id
advertiser_id
app_id
campaign_id
country_code
game_feed_id
game_template_id
media_app_id
os_version
uid
video_template_id
mst_advertiser_order_id


# fe

In [7]:
train["type"] = "train"
test["type"] = "test"
all_df = pd.concat([train, test])

In [8]:
tmp = all_df.uid.value_counts().reset_index(drop=False)
multi_users = list(tmp[tmp.uid>=2]["index"])

In [9]:
all_df = all_df[all_df.uid.isin(multi_users)].reset_index(drop=True)

In [10]:
# cumsum from click by uid
tmp1 = all_df.groupby(['uid'])['from_click'].progress_apply(lambda x: pd.DataFrame([0]+list(x.cumsum()[:-1]))).reset_index(drop=True).values
tmp1 = np.array(tmp1, dtype=float)
# cumsum total num by uid
tmp2 = all_df.groupby(['uid']).cumcount().values
tmp2 = np.array(tmp2, dtype=float).reshape(-1,1)
# https://stackoverflow.com/questions/26248654/how-to-return-0-with-divide-by-zero
from_click_ratio = np.divide(tmp1, tmp2, out=np.ones_like(tmp1)*-1, where=tmp2!=0)
all_df["click_ratio"] = from_click_ratio

In [11]:
train = train.merge(all_df[["request_id","click_ratio"]], on="request_id", how="left")
test = test.merge(all_df[["request_id","click_ratio"]], on="request_id", how="left")

In [12]:
train["click_ratio"] = train["click_ratio"].fillna(-1)
test["click_ratio"] = test["click_ratio"].fillna(-1)

In [13]:
train.drop(['request_id', 'type'], axis=1, inplace=True)
test.drop(['request_id', 'type'], axis=1, inplace=True)

In [14]:
train.head()

,imp_at,target,adnw_id,adspot_id,adspot_video_format_id,advertiser_id,app_id,auction_type_id,campaign_id,category_id,...,media_app_id,os,os_version,pos,uid,user_type_id,video_template_id,mst_advertiser_order_id,mst_user_type_id,click_ratio
0,2020-04-28 00:04:14.639,0,4,1111,2.0,20,14872,1.0,403,8497,...,0,1,87,0,273693,1,6,37,35,-1.0
1,2020-04-28 00:09:13.759,0,28,1081,2.0,121,8301,2.0,91,0,...,8995,2,152,1,427139,2,4,166,37,-1.0
2,2020-04-28 00:09:14.708,0,42,1123,3.0,5,12948,1.0,114,0,...,347,1,55,0,1005266,1,0,27,35,-1.0
3,2020-04-28 00:10:40.291,0,8,1113,2.0,22,12379,1.0,75,0,...,6145,1,91,0,492541,1,7,159,38,-1.0
4,2020-04-28 00:11:45.895,0,22,1061,NaN,135,196,4.0,331,0,...,2190,1,71,0,698493,1,0,56,35,-1.0


# separation for validation

In [15]:
mask = train.imp_at > "2020-05-21"
valid = train.loc[mask].copy()
train = train.loc[~mask]

In [16]:
remove_features = ["imp_at","game_feed_asset_type_id","country_code","pos","video_template_id"]
    
train = train.drop(remove_features, axis=1)
valid = valid.drop(remove_features, axis=1)
test = test.drop(remove_features, axis=1)

In [17]:
train.shape, valid.shape

((1568715, 28), (428880, 28))

In [18]:
train.head()

,target,adnw_id,adspot_id,adspot_video_format_id,advertiser_id,app_id,auction_type_id,campaign_id,category_id,first_login_interval,...,login_frequency,max_login_interval,media_app_id,os,os_version,uid,user_type_id,mst_advertiser_order_id,mst_user_type_id,click_ratio
0,0,4,1111,2.0,20,14872,1.0,403,8497,1490644.0,...,10.0,2607.0,0,1,87,273693,1,37,35,-1.0
1,0,28,1081,2.0,121,8301,2.0,91,0,790412.0,...,2.0,13540.0,8995,2,152,427139,2,166,37,-1.0
2,0,42,1123,3.0,5,12948,1.0,114,0,1506105.0,...,9.0,33725.0,347,1,55,1005266,1,27,35,-1.0
3,0,8,1113,2.0,22,12379,1.0,75,0,184283.0,...,4.0,7377.0,6145,1,91,492541,1,159,38,-1.0
4,0,22,1061,NaN,135,196,4.0,331,0,1999495.0,...,5.0,164759.0,2190,1,71,698493,1,56,35,-1.0


# modelling

In [19]:
object_feats = [i for i in object_feats if i not in remove_features and i != "request_id"]

In [20]:
lgbm_params = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt', 
               'tree_learner': 'serial', 'learning_rate': 0.1, "num_leaves": 10, 'random_seed':44,'max_depth': 5}

def pr_auc_metric(y_predicted, y_true):
    return 'pr_auc', average_precision_score(y_true.get_label(), y_predicted), True

y_train = train.target.copy()
X_train = train.drop(['target'],axis=1).copy()
y_valid = valid.target.copy()
X_valid = valid.drop(['target'],axis=1).copy()
    
valid_lgb = pd.DataFrame(np.zeros([X_train.shape[0]]))
real = np.array([])
features_list = [i for i in X_train.columns]
feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
        
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
        
model = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
       num_boost_round=10000,early_stopping_rounds=20,verbose_eval = 10, feval=pr_auc_metric, categorical_feature = sorted(object_feats))
        
valid_predict = model.predict(X_valid, num_iteration = model.best_iteration)
feature_importance_df["Importance"] = model.feature_importance()
        
score = average_precision_score(y_valid, valid_predict)
print("average precision score = {}".format(score))
pred_value = model.predict(test, num_iteration = model.best_iteration)

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['advertiser_id', 'app_id', 'campaign_id', 'game_feed_id', 'game_template_id', 'media_app_id', 'mst_advertiser_order_id', 'os_version', 'uid']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 20 rounds
[10]	training's pr_auc: 0.215348	valid_1's pr_auc: 0.178713
[20]	training's pr_auc: 0.225831	valid_1's pr_auc: 0.186337
[30]	training's pr_auc: 0.236239	valid_1's pr_auc: 0.193677
[40]	training's pr_auc: 0.246106	valid_1's pr_auc: 0.199107
[50]	training's pr_auc: 0.251361	valid_1's pr_auc: 0.201729
[60]	training's pr_auc: 0.256651	valid_1's pr_auc: 0.203636
[70]	training's pr_auc: 0.261898	valid_1's pr_auc: 0.20547
[80]	training's pr_auc: 0.265297	valid_1's pr_auc: 0.206553
[90]	training's pr_auc: 0.268574	valid_1's pr_auc: 0.207542
[100]	training's pr_auc: 0.272053	valid_1's pr_auc: 0.208604
[110]	training's pr_auc: 0.27449	valid_1's pr_auc: 0.209027
[120]	training's pr_auc: 0.2762	valid_1's pr_auc: 0.207758
[130]	training's pr_auc: 0.278085	valid_1's pr_auc: 0.208109
Early stopping, best iteration is:
[118]	training's pr_auc: 0.275957	valid_1's pr_auc: 0.209407
average precision score = 0.2094074255815394


In [21]:
feature_importance_df.sort_values("Importance", ascending=False)

,Feature,Importance
11,game_feed_id,201
4,app_id,197
6,campaign_id,189
22,uid,116
19,media_app_id,73
3,advertiser_id,48
24,mst_advertiser_order_id,37
8,first_login_interval,27
17,login_frequency,26
16,last_login_interval,25


# submission

In [22]:
lgb_sub = pd.read_csv(DIR + "atmaCup7__sample_submission.csv")
lgb_sub["target"] = pred_value
lgb_sub.to_csv("atmacup7_lgb"+str(score)[:-10]+".csv", index = False)